© Anish Anand 
# In this lab , we are going to work on CIFAR10 small images classification dataset.

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
 Link to dataset to read more : https://www.cs.toronto.edu/~kriz/cifar.html

## Important feature of the dataset 

1. It has 10 classes  [airplane
,automobile
,bird
,cat
,deer
,dog
,frog
,horse
,ship
,truck]

2. consists of 60000 32x32 colour images

3. No. of training images : 50000

4. No. of testing images 10000

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

In [32]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

Now we are loading the dataset by just referring the load data function. Since the dataset belongs to Keras so we have already a function to load this dataset.

In [33]:
x=keras.datasets.cifar10.load_data()

In [34]:
print(type(x))

<class 'tuple'>


In [35]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [36]:
(x_train, y_train), (x_test, y_test) = x

In [37]:
x_train.shape

(50000, 32, 32, 3)

In [38]:
y_train.shape

(50000, 1)

In [39]:
x_test.shape

(10000, 32, 32, 3)

In [40]:
y_test.shape

(10000, 1)

So till now we can conclude the following points:

1. x_train and x_test are 50000 and 10000 images respectively whose sizes are 32 X 32 and consist the third dimension as RGB therefore 3.

2. y_train and y_test are 50000 and 10000 images respectively which has information about the class in which they belong.

3. We have 10 classes and we are training at least 10000 images to each class and nthen predicting it right.

## 1.Preprocessing the images

We are scaling images so that there must not be any ambiguity.

In [41]:
X_train_scaled = x_train/255
X_test_scaled = x_test/255

#### Ques : Why we divide with 255?

#### Ans: Because each and every images present in training and testing dataset has range from 0 to 255 because RGB ranges the same. Dividing with the highest value we are actually normalising the matrix values so the model can perform well.

## 2.Performing One hot encoding (part of preprocessing if images)

One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction. For example if we have 3 classes and we have one image which lies in the 2nd class then we can represent this information as [0,1,0]. This is what we have done One Hot Encoding.

SO here we have 10 classes which means that when any image given belongs to 2nd class then we can represent this as [0,1,0,0,0,0,0,0,0,0].

Luckily we dont not need to create list full of zero everytime and adjust values accordingly , istead of that we can do this by using an API which keras has keras.utils.to_categorical().

In [42]:
#doing one hot encoding 

y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=10, dtype='float32'
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=10, dtype='float32'
)

In [43]:
y_train[0:5] 

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [44]:
y_train_categorical[0:5]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

Outpit 19 and output 20 just telling us the difference between image classes after one hot encoding.

## 3. Model Building now 

In [45]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(32,32,3)),
    keras.layers.Dense(3000,activation='relu'),
    keras.layers.Dense(1000,activation='relu'),
    keras.layers.Dense(10,activation='sigmoid')
])

#we have built our neural network so far
#we have built just Artificial Neural Network 

#compiling the model

model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train_categorical, epochs=1)

1563/1563 [==============================] - 13s 8ms/step - loss: 1.8113 - accuracy: 0.3535


## 4. Making prediction now





In [47]:
#lets predict the first image from test dataset

model.predict(X_test_scaled)[0]

array([0.37141418, 0.52542984, 0.7199892 , 0.7889977 , 0.3919865 ,
       0.6188356 , 0.6664617 , 0.14196117, 0.5393762 , 0.17657457],
      dtype=float32)

In [50]:
#in above it is probability of each class and we actually use np.argmax function to filter the maximum probability

np.argmax(model.predict(X_test_scaled)[0])

3

In [52]:
#predicting class 3, lets see what is in class[3]

classes[np.argmax(model.predict(X_test_scaled)[0])]

'cat'

In [57]:
#now lets check what actually the first image is

y_test[0]

array([3], dtype=uint8)

In [58]:
classes[y_test[0][0]]

'cat'

So far we saw that we have built our model and it predicting correctly.

### Now our task would be to compare the performance of GPU and CPU.

In [59]:
#first of all I am making a copy of my model and will call it when needed 
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')    
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

## Measuring training time on CPU 

In [60]:
%%timeit -n1 -r1 #measures time taken by a cell to run

with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(X_train_scaled, y_train_categorical, epochs=1)

1563/1563 [==============================] - 167s 106ms/step - loss: 1.8135 - accuracy: 0.3534
2min 57s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Measuring training time on GPU

In [61]:
%%timeit -n1 -r1
with tf.device('/GPU:0'):
    cpu_model = get_model()
    cpu_model.fit(X_train_scaled, y_train_categorical, epochs=1)
    

1563/1563 [==============================] - 13s 8ms/step - loss: 1.8064 - accuracy: 0.3542
15.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Just for 1 epoch we have seen the GPU is almost 7 times faster than CPU

## Repeating the same process for 10 epochs

### CPU

In [62]:
%%timeit -n1 -r1 #measures time taken by a cell to run

with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(X_train_scaled, y_train_categorical, epochs=10)

Epoch 1/10
1563/1563 [==============================] - 168s 107ms/step - loss: 1.8140 - accuracy: 0.3533
Epoch 2/10
1563/1563 [==============================] - 164s 105ms/step - loss: 1.6253 - accuracy: 0.4259
Epoch 3/10
1563/1563 [==============================] - 171s 110ms/step - loss: 1.5443 - accuracy: 0.4579
Epoch 4/10
1563/1563 [==============================] - 181s 116ms/step - loss: 1.4813 - accuracy: 0.4772
Epoch 5/10
1563/1563 [==============================] - 144s 92ms/step - loss: 1.4341 - accuracy: 0.4940
Epoch 6/10
1563/1563 [==============================] - 139s 89ms/step - loss: 1.3901 - accuracy: 0.5123
Epoch 7/10
1563/1563 [==============================] - 137s 87ms/step - loss: 1.3525 - accuracy: 0.5243
Epoch 8/10
1563/1563 [==============================] - 136s 87ms/step - loss: 1.3154 - accuracy: 0.5400
Epoch 9/10
1563/1563 [==============================] - 136s 87ms/step - loss: 1.2853 - accuracy: 0.5484
Epoch 10/10
1563/1563 [============================

### GPU

In [63]:
%%timeit -n1 -r1
with tf.device('/GPU:0'):
    cpu_model = get_model()
    cpu_model.fit(X_train_scaled, y_train_categorical, epochs=10)

Epoch 1/10
1563/1563 [==============================] - 13s 8ms/step - loss: 1.8144 - accuracy: 0.3549
Epoch 2/10
1563/1563 [==============================] - 12s 7ms/step - loss: 1.6237 - accuracy: 0.4260
Epoch 3/10
1563/1563 [==============================] - 12s 8ms/step - loss: 1.5404 - accuracy: 0.4551
Epoch 4/10
1563/1563 [==============================] - 12s 8ms/step - loss: 1.4815 - accuracy: 0.4777
Epoch 5/10
1563/1563 [==============================] - 12s 8ms/step - loss: 1.4308 - accuracy: 0.4971
Epoch 6/10
1563/1563 [==============================] - 12s 8ms/step - loss: 1.3876 - accuracy: 0.5120
Epoch 7/10
1563/1563 [==============================] - 12s 8ms/step - loss: 1.3512 - accuracy: 0.5257
Epoch 8/10
1563/1563 [==============================] - 12s 8ms/step - loss: 1.3162 - accuracy: 0.5397
Epoch 9/10
1563/1563 [==============================] - 12s 8ms/step - loss: 1.2826 - accuracy: 0.5482
Epoch 10/10
1563/1563 [==============================] - 12s 8ms/step - l

## Result : GPU is almost 12 times faster than CPU.